In [36]:
import h5py
import tensorflow as tf
import numpy as np
import math
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models, Model, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda

In [70]:
hdf5_file = "energy_data.h5"

In [72]:
def hdf5_generator():
    with h5py.File(hdf5_file, 'r') as f:
        data = f['data']
        labels = f['labels']
        num_samples = data.shape[0]
        
        for i in range(num_samples):
            yield data[i], labels[i]

In [74]:
def _fixup_shape(images, labels):
    images.set_shape([32, 20, 110, 11])
    labels.set_shape([32,])
    return images, labels

In [76]:
dataset = tf.data.Dataset.from_generator(
    hdf5_generator,
    output_signature=(
        tf.TensorSpec(shape=(20, 110, 11), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int16)
    )
)
# Optimize the pipeline
dataset = (dataset
           .batch(32,drop_remainder=True)
           .map(_fixup_shape)
           .prefetch(tf.data.AUTOTUNE))
print("Data loaded")

Data loaded


In [12]:
input_3d = Input(shape=(20, 110, 11, 1), name='3D_Input')

# 3D CNN Path
x = layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu')(input_3d)  # 'same' padding to preserve dimensions
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

x = layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

# Flatten 3D features
x = layers.Flatten()(x)

# Fully Connected Layers
fc = layers.Dense(128, activation='relu')(x)
fc = layers.Dense(64, activation='relu')(fc)

# Output Layer
output = layers.Dense(1, activation='linear')(fc)

# Define the Model
model = Model(inputs=[input_3d], outputs=output)

# Compile the Model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 3D_Input (InputLayer)           │ (None, 20, 110, 11, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 18, 108, 9, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 9, 54, 4, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 9, 54, 4, 32)   │        55,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 4, 27, 2, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       884,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 950,305 (3.63 MB)

 Trainable params: 950,305 (3.63 MB)

 Non-trainable params: 0 (0.00 B)

In [78]:
with h5py.File(hdf5_file, "r") as hdf:
    dataset_size = len(hdf["data"])  # Or hdf["labels"], if they have the same length
    print(f"Dataset size: {dataset_size}")

Dataset size: 450000


In [80]:
p=0.001
taken_size=p*dataset_size
val_dataset=dataset.take(int(0.1*dataset_size*p))
train_dataset=dataset.skip(int(0.1*p*dataset_size)).take(int(0.9*dataset_size*p))

In [82]:
early_stopping = EarlyStopping(
    monitor='val_loss',         # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
    patience=3,                 # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True   # Restore model weights from the best epoch
)

In [22]:
print("Training")
#Trening
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    shuffle=True,
    epochs=20,                 # Set a high number of epochs; EarlyStopping will stop early
    callbacks=[early_stopping],  # Add EarlyStopping to callbacks
    verbose=1
)
print("Training done")

Training
Epoch 1/20


C:\Users\spbki\AppData\Local\anaconda3\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['3D_Input']
Received: inputs=Tensor(shape=(32, 20, 110, 11))
  warnings.warn(msg)


     19/Unknown 8s 290ms/step - loss: 8883.1816 - mean_absolute_error: 77.7519

KeyboardInterrupt: 

In [84]:
test_hdf5_file = "angle_data.h5"

In [86]:
def test_hdf5_generator():
    with h5py.File(test_hdf5_file, 'r') as f:
        data = f['data']
        labels = f['labels']
        num_samples = data.shape[0]
        
        for i in range(num_samples):
            yield data[i], labels[i]

In [88]:
def _fixup_shape(images, labels):
    images.set_shape([32, 20, 110, 11])
    labels.set_shape([32,])
    return images, labels

In [90]:
test_dataset = tf.data.Dataset.from_generator(
    test_hdf5_generator,
    output_signature=(
        tf.TensorSpec(shape=(20, 110, 11), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int16)
    )
)
# Optimize the pipeline
test_dataset = (test_dataset
           .batch(32,drop_remainder=True)
           .map(_fixup_shape)
           .prefetch(tf.data.AUTOTUNE))
print("Data loaded")

Data loaded


In [92]:
test_ds=test_dataset.take(int(int(0.1*dataset_size*p)))

In [77]:
test_loss, test_mae = model.evaluate(test_ds)
print("Test MAE dla sieci konwolucyjnej:", test_mae)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 42.0869 - mean_absolute_error: 4.9855
Test MAE dla sieci konwolucyjnej: 4.908227443695068


In [94]:
input_3d = Input(shape=(20, 110, 11, 1), name='3D_Input')

# 3D CNN Path
x = layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu')(input_3d) 
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

x = layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

# Flatten 3D features
x = layers.Flatten()(x)

summed_tensor = Lambda(lambda t: tf.reduce_sum(t, axis=[1, 2, 3, 4]))(input_3d)  # Sum over spatial dimensions
summed_tensor_expanded = Lambda(lambda t: tf.expand_dims(t, axis=-1))(summed_tensor)  # Expand to (None, 1)

# Ensure shapes are compatible for concatenation
combined = layers.Concatenate(axis=-1)([x, summed_tensor_expanded])  # Concatenate along last axis


# Fully Connected Layers
fc = layers.Dense(128, activation='relu')(combined)
fc = layers.Dense(64, activation='relu')(fc)

# Output Layer
output = layers.Dense(1, activation='linear')(fc)

# Define the Model
model = Model(inputs=[input_3d], outputs=output)

# Compile the Model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

#do przetestowania, w teorii powinien brać też pod uwagę całkowitą energię.

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ 3D_Input            │ (None, 20, 110,   │          0 │ -                 │
│ (InputLayer)        │ 11, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_8 (Conv3D)   │ (None, 18, 108,   │      1,792 │ 3D_Input[0][0]    │
│                     │ 9, 64)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_8     │ (None, 9, 54, 4,  │          0 │ conv3d_8[0][0]    │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_9 (Conv3D)   │ (None, 9, 54, 4,  │     55,328 │ max_pooling3d_8[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_9     │ (None, 4, 27, 2,  │          0 │ conv3d_9[0][0]    │
│ (MaxPooling3D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None)            │          0 │ 3D_Input[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 6912)      │          0 │ max_pooling3d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 1)         │          0 │ lambda_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 6913)      │          0 │ flatten_4[0][0],  │
│ (Concatenate)       │                   │            │ lambda_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │    884,992 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 64)        │      8,256 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         65 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 950,433 (3.63 MB)

 Trainable params: 950,433 (3.63 MB)

 Non-trainable params: 0 (0.00 B)

In [96]:
print("Training")
#Trening
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    shuffle=True,
    epochs=20,                 # Set a high number of epochs; EarlyStopping will stop early
    callbacks=[early_stopping],  # Add EarlyStopping to callbacks
    verbose=1
)
print("Training done")

Training
Epoch 1/20
    405/Unknown 113s 272ms/step - loss: 969.5482 - mean_absolute_error: 14.6882

C:\Users\spbki\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


405/405 ━━━━━━━━━━━━━━━━━━━━ 116s 279ms/step - loss: 967.7225 - mean_absolute_error: 14.6705 - val_loss: 45.4032 - val_mean_absolute_error: 5.0920
Epoch 2/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 110s 270ms/step - loss: 51.1728 - mean_absolute_error: 5.4278 - val_loss: 44.7395 - val_mean_absolute_error: 5.0356
Epoch 3/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 110s 269ms/step - loss: 50.8705 - mean_absolute_error: 5.4114 - val_loss: 44.4245 - val_mean_absolute_error: 5.0004
Epoch 4/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 110s 269ms/step - loss: 50.1640 - mean_absolute_error: 5.3778 - val_loss: 44.1034 - val_mean_absolute_error: 4.9912
Epoch 5/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 111s 270ms/step - loss: 49.7274 - mean_absolute_error: 5.3577 - val_loss: 44.5286 - val_mean_absolute_error: 5.0035
Epoch 6/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 110s 270ms/step - loss: 49.4014 - mean_absolute_error: 5.3391 - val_loss: 43.8917 - val_mean_absolute_error: 4.9718
Epoch 7/20
405/405 ━━━━━━━━━━━━━━━━━━━━ 111s 270ms/step - loss: 48.8615 -

In [98]:
test_loss, test_mae = model.evaluate(test_ds)
print("Test MAE dla sieci konwolucyjnej:", test_mae)

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 30.3673 - mean_absolute_error: 4.2702
Test MAE dla sieci konwolucyjnej: 4.221895217895508
